6/16(목) 미팅 내용 정리
  발표 자료 작성, 포함 내용 (아래 참고)
   1차 취합  : 6/17 16시까지.
   2차 취합 : 6/20 15시 까지.

<작업 내용>
1. 전처리(개인 별 작성)
   Feature 선택.
   Feature 가공.
2. hyper parameter 튜닝 (개인 별 작성)
3. 모델별 성능 비교.

## 1. 문제 정의

PUBG사로부터 제공받은 다양한 데이터를 통해 사용자가 몇 등으로 게임을 종료하게 될 지 예측해본다. 


## 2. 데이터 수집

분석에 필요한 데이터 불러오기


## 3. 데이터 탐색
### EDA
1. 박스플롯으로 전체적인 데이터 분석
2. feature별 상관관계 비교
3. 각 feature별 데이터 분포, 특징 확인




## 4. 데이터 전처리
1. 'winPlacePerc'컬럼에서 1개의 결측치 제거  
2. 이상치 제거
3. 경기 종류에 따른 분리, kill/rank/win point 변환  >>>  one-hot-encode
4. feature 선택 


## 5. 모델링
1. LGBMRegressor / mean_absolute_error 사용

## #결과 및 중요 데이터 정리

In [ ]:
# outlier 제거에 사용
train_main = train_main[(((train_main['winPlacePerc']+0.5) / (train_main['walkDistance'])) > 1/13000) & (train_main['walkDistance']>1) ]
train_main = train_main[((train_main['winPlacePerc'] / (train_main['killPlace']-103)) > -1/42)]
train_main = train_main[((train_main['winPlacePerc']+ 0.51 ) / (train_main['boosts'])) > 1/17 ]
train_main = train_main[((train_main['winPlacePerc']+0.5 ) / (train_main['heals'])) > 1/50 ]
train_main = train_main[((train_main['winPlacePerc']+0.8 ) / (train_main['kills'])) > 1/30]
train_main = train_main[((train_main['winPlacePerc']+1.7 ) / (train_main['killStreaks'])) > 1/6]
train_main = train_main[((train_main['winPlacePerc']+ 0.75) / (train_main['longestKill'])) > 1/600]
train_main = train_main[((train_main['winPlacePerc']+0.73 ) / (train_main['rideDistance'])) > 1/24000]

train_sub = train_sub[((train_sub['winPlacePerc']+ 3.8 ) / (train_sub['weaponsAcquired'])) > 1/20 ]
train_sub = train_sub[((train_sub['winPlacePerc']+ 0.25 ) / (train_sub['headshotKills'])) > 1/40 ]
train_sub = train_sub[((train_sub['winPlacePerc']+ 2.8 ) / (train_sub['assists'])) > 1/5 ]
train_sub = train_sub[((train_sub['winPlacePerc']+ 1.25 ) / (train_sub['DBNOs'])) > 1/16 ]

### 전처리에 따른 비교_ killPlace 포함

|결측치 제거|아웃라이어 제거|매치 타입 분류|kill/rank/winPoint 변환|결과값(MAE)|비고|
|---|---|---|---|---|---|
| X | X | X | X |  <span style='color:blue'> 0.0604 </span>  |<span style='color:green'>  obiect 컬럼 제거 </span>|
| O | X | X | X |  <span style='color:red'> 0.0603 </span>  |<span style='color:green'>  obiect 컬럼 제거 </span>|
| O | O | X | X |  <span style='color:blue'> 0.0609 </span>  |<span style='color:green'>  obiect 컬럼 제거 </span>|
| O | O | O | X |  <span style='color:blue'> 0.0689 </span>  ||
| O | O | O | O |  <span style='color:blue'> 0.0689 </span>  ||


### 전처리에 따른 비교_ killPlace 제외

|결측치 제거|아웃라이어 제거|매치 타입 분류|kill/rank/winPoint 변환|결과값(MAE)|비고|
|---|---|---|---|---|---|
| X | X | X | X |  <span style='color:blue'> 0.09078 </span>  |<span style='color:green'>  obiect 컬럼 제거 </span>|
| O | X | X | X |  <span style='color:red'> 0.09070 </span>  |<span style='color:green'>  obiect 컬럼 제거 </span>|
| O | O | X | X |  <span style='color:blue'> 0.0912 </span>  |<span style='color:green'>  obiect 컬럼 제거 </span>|
| O | O | O | X |  <span style='color:blue'> 0.0995 </span>  ||
| O | O | O | O |  <span style='color:blue'> 0.0995 </span>  ||

### feature에 따른 비교_ killPlace 포함

|main feature|sub feature|object feature|결과값(MAE)|비고|
|--|--|--|--|--|
|O|X|X|<span style='color:blue'> 0.0740 </span>||
|O|O|X|<span style='color:blue'> 0.0720 </span>||
|O|X|O|<span style='color:blue'> 0.0693 </span>||
|O|O|O|<span style='color:red'> 0.0689 </span>||


### feature에 따른 비교_ killPlace 제외

|main feature|sub feature|object feature|결과값(MAE)|비고|
|--|--|--|--|--|
|O|X|X|<span style='color:blue'> 0.1077 </span>||
|O|O|X|<span style='color:blue'> 0.1046 </span>||
|O|X|O|<span style='color:blue'> 0.1009 </span>||
|O|O|O|<span style='color:red'> 0.0995 </span>||


<span style='color:green'>단, 모든 전처리 과정을 거친 데이터를 사용</span>

- main feature

walkDistance, killPlace, boosts, heals, kills, killStreaks, longestKill, rideDistance

- sub feature

weaponsAcquired, damageDealt, headshotKills, assists, DBNOs

- object feature

matchTypes (killPoints, rankPoints, winPoints 사용)



### hyper parameter 비교

<span style='color:green'>단, 모든 전처리 과정을 거친 데이터를 사용</span>
#### 주요 hp
1. max_depth=- 1 (가장 민감하기 때문에 가장 먼저 튜닝해야됨! ,보통 3~12, 클수록 피팅 잘되지만 오버피팅 가능성도 up)
- max_depth = 1 : 0.0955  / killPlace제외: 0.1067
- max_depth = 5 : 0.0698 
- ...
- max_depth = 9 : 0.0690  / killPlace제외: 0.09955
- <span style='color:red'> max_depth = 10 일 때 : 0.0689</span> / killPlace제외: 0.099533  
- max_depth = 11 / killPlace제외: <span style='color:red'>0.099532</span>
- max_depth = 12 : 0.0689  / killPlace제외: 0.099538
- max_depth = 20 : 0.0689
- max_depth = 50 : 0.0689


2. num_leaves = 31 2번째로 민감
- num_leaves = 5 : 0.0755
- num_leaves = 10 : 0.0715
- num_leaves = 30 : 0.0690
- num_leaves = 40 : 0.0687 / killPlace제외: 0.0994
- <span style='color:red'> num_leaves = 100 : 0.0681 / killPlace제외: 0.0991</span>
- num_leaves = 150 : 0.0.0679  / killPlace제외: 0.0991
- num_leaves = 300 : 0.0677
- num_leaves = 500 : 0.0676
- num_leaves = 1000 : 0.0676


3. colsample_bytree 3번째로 민감, 보통 0.7로 설정
- colsample_bytree=0.4 : 0.0699
- colsample_bytree=0.6 : 0.0681
- <span style='color:red'> colsample_bytree=0.7 : 0.0680 </span>/ killPlace제외: 0.0992 
- colsample_bytree=0.8 : 0.0680  / killPlace제외:<span style='color:red'>0.0991663</span>
- colsample_bytree=0.9:  / killPlace제외:0.09916510
- colsample_bytree=1   : 0.0681 (default)/ killPlace제외: 0.09916508




4. learning_rate 보통 작을수록 모형 성능 향상, 보통 0.1~0.3, 최종 모형 학습시 0.05 이하 추천
- learning_rate=0.05 : 0.0692  / killPlace제외: 0.0997
- learning_rate=0.08 :   / killPlace제외: 0.0992
- learning_rate=0.09 :   / killPlace제외: 0.09919
- learning_rate=0.1 : 0.0680 (defalut) / killPlace제외:<span style='color:red'>0.09916</span>
- <span style='color:red'> learning_rate=0.3 : 0.0679</span> / killPlace제외: 0.09918
- learning_rate=0.5 : 0.0680




5. n_estimators 적합(fit)시킬 boosted tree의 수
- n_estimators= 50 : 0.0682
- n_estimators=100 : 0.0679 (default)  / killPlace제외: 0.9916
- <span style='color:red'> n_estimators=500 : 0.0677 / killPlace제외: 0.09896 </span>
- n_estimators=1000 : 0.0678 / killPlace제외: 0.09897

#### 그 외 hp

왠만하면 건들지 않는게 좋음
- boosting_type (default = 'gbdt')
- subsample_for_bin=200000
- objective=None
- class_weight=None
- min_split_gain=0.0
- min_child_weight=0.001
- min_child_samples=20
- subsample=1.0
- subsample_freq=0
- reg_alpha=0.0
- reg_lambda=0.0
- random_state=None
- n_jobs=- 1
- importance_type='split'

## 최적의 결과 : 
### 1. 결측치만 제거(+object 컬럼 제거)
### 2. EDA로 선별된 feature들을 모두 사용
### 3. hyper-parameter를 적용한 모델

### >> 결과: 0.0568  / killPlace제외: 0.1033

+아웃라이어: 0.0576

+피쳐 정리(main + sub feature): 0.0661

+(main + sub feature) - weaponsAcquired : 0.0665

feature를 줄일수록 더 안좋아짐!